In [1]:
import numpy as np

from qiskit import QuantumCircuit

from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit.library import RealAmplitudes, UnitaryGate

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke,FakeBrisbane

from qiskit_aer import AerSimulator

import time

from scipy.optimize import minimize

import matplotlib.pyplot as plt

In [2]:
# backend = FakeSherbrooke()
# backend = FakeBrisbane()
backend = AerSimulator(method='statevector')

In [3]:
def reference_preperation(num_qubits, i, unitary_matrix):
    """
    Prepare the i-th reference state for SSVQE     
    """
    u_gate = UnitaryGate(unitary_matrix, label='initial_guess')

    reference_circuit = QuantumCircuit(num_qubits)

    binary_index = np.binary_repr(i, num_qubits)
    for j in range(num_qubits):
        if binary_index[-j-1] == '1':
            reference_circuit.x(j)

    reference_circuit.append(u_gate, list(range(num_qubits)))
    
    return reference_circuit

In [4]:
def cost_func(params, ansatz_list: list, hamiltonian_list: list, weighting: list, estimator: Estimator, callback_dict):
    """Return callback function that uses Estimator instance,
    and stores intermediate values into a dictionary.

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance
        callback_dict (dict): Mutable dict for storing values

    Returns:
        Callable: Callback function object
    """
    pubs = []
    for ansatz, hamiltonian in zip(ansatz_list, hamiltonian_list):
        pubs.append([ansatz, [hamiltonian], [params]])

    job = estimator.run(pubs=pubs)
    result = job.result()

    callback_dict["job_ids"].append(job.job_id())

    cost = 0
    energies = []
    for i in range(len(ansatz_list)):
        energies.append(float(result[i].data.evs[0]))
        cost += weighting[i] * result[i].data.evs[0]
    
    # Keep track of the number of iterations
    callback_dict["iters"] += 1
    # Set the prev_vector to the latest one
    callback_dict["prev_vector"] = params
    # Compute the value of the cost function at the current vector
    callback_dict["cost_history"].append(cost)
    # Keep trck of the energy expetation values of diferent reference states
    callback_dict["energies_history"].append(energies)
    # Grab the current time
    current_time = time.perf_counter()
    # Find the total time of the execute (after the 1st iteration)
    if callback_dict["iters"] > 1:
        callback_dict["_total_time"] += current_time - callback_dict["_prev_time"]
    # Set the previous time to the current time
    callback_dict["_prev_time"] = current_time
    # Compute the average time per iteration and round it
    time_str = (
        round(callback_dict["_total_time"] / (callback_dict["iters"] - 1), 2)
        if callback_dict["_total_time"]
        else "-"
    )
    # Print to screen on single line

    print(f"Iters. done: {callback_dict['iters']} [Current cost: {cost}]")

    return cost
    

In [5]:
def run_ssvqe(initial_parameters, ansatz_list, operator_list, weighting, estimator, method):
    callback_dict = {
        "prev_vector": None,
        "iters": 0,
        "job_ids": [],
        "cost_history": [],
        "energies_history": [],
        "_total_time": 0,
        "_prev_time": None,
    }
   
    result = minimize(
        cost_func,
        initial_parameters,
        args=(ansatz_list, operator_list, weighting, estimator, callback_dict),
        method=method,
    )
    return result, callback_dict

In [8]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

num_qubits = 2
num_states = 4
num_layers = 2

weighting = [4, 3, 2, 1]
method = 'COBYLA'

var_form = RealAmplitudes(num_qubits=num_qubits, entanglement='linear', reps=num_layers, insert_barriers=True)
num_params = var_form.num_parameters

initial_guess = 0.5 * np.asarray([[ 1,  1,  1,  1],
                                  [ 1,  1, -1, -1],
                                  [ 1, -1, -1,  1],
                                  [ 1, -1,  1, -1]])

initial_parameters = np.zeros(num_params)
# initial_parameters = 2 * np.pi * np.random.rand(num_params)
# initial_parameters = [1.35140561, 5.25284732, 3.27915769, 5.62247295, 1.55055985, 5.90935563, 1.62797683, 0.19680399, 5.26257616, 0.39043692]
# initial_parameters = [1.09069916, 2.70357009, 3.30657582, 3.26453662, 1.00818528, 2.69149415, 1.86558426, 4.72284122] # cyclobutadiene guess01
# initial_parameters = [2.33806231, 6.27685914, 4.811418  , 2.01502319, 2.00193456, 6.17161626, 5.16353018, 4.05523528] # cyclobutadiene guess02
# initial_parameters = [-0.13258053, -0.18639556,  0.03058726,  0.35835309,  0.08911584,  0.11449962]

ansatz_list = []
for index in range(num_states):
    ansatz = reference_preperation(num_qubits, index, initial_guess).compose(var_form)
    ansatz_list.append(ansatz)

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
transpiled_ansatz_list = pm.run(ansatz_list)

H = [[ 0, -1,  0,  0],
     [-1,  0, -1,  0],
     [ 0, -1,  0, -1],
     [ 0,  0, -1,  0]]
# pauli_op = SparsePauliOp(['IIX', 'IXX', 'IYY', 'XXX', 'XYY', 'YYX', 'YXY', 'XIX', 'XZX', 'YIY', 'YZY'], coeffs=[1, 0.5, 0.5, 0.25, -0.25, 0.25, 0.25, 0.25, 0.25, -0.25, -0.25])
# pauli_op = SparsePauliOp(['IX', 'XX', 'YY'], coeffs=[-1, -0.5, -0.5]) # 1,3-butadiene
# pauli_op = SparsePauliOp(['IX', 'XX'], coeffs=[-1, -1]) # cyclobutadiene
pauli_op = SparsePauliOp.from_operator(H)

transplied_op_list = []
for index in range(num_states):
    transplied_op_list.append(pauli_op.apply_layout(transpiled_ansatz_list[index].layout))
style = {
    "figwidth": 256
}
transpiled_ansatz_list[3].draw(output='mpl', idle_wires=False, style=style)

ValueError: Image size of 262704x43674 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 262704x43674.6 with 1 Axes>

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000
    
    vqe_result, callback_dict = run_ssvqe(
        initial_parameters=initial_parameters,
        ansatz_list=transpiled_ansatz_list,
        weighting=weighting,
        operator_list=transplied_op_list,
        estimator=estimator,
        method=method,
    )

file_name = "Huckel_SSVQE_simu_020.txt"

with open(file=file_name, mode='xt', encoding='UTF-8') as file:
    print("Backend: {}\n".format(backend), file=file)
    print("-------------------------------------------------", file=file)
    print("initial_guess:\n{}\n".format(initial_guess), file=file)
    print("initial_parameters:\n{}\n".format(initial_parameters), file=file)
    print("number_layers: {}\n".format(num_layers), file=file)
    print("Hamiltonian =\n{}\n".format(pauli_op), file=file)
    print("-------------------------------------------------", file=file)
    print("Cost history:\n{}\n".format(callback_dict["cost_history"]), file=file)
    print("-------------------------------------------------", file=file)
    print("Energies history:\n{}\n".format(callback_dict["energies_history"]), file=file)

In [ ]:
fig, ax = plt.subplots()
plt.plot(range(callback_dict["iters"]), callback_dict["cost_history"])
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.draw()

print(callback_dict["cost_history"][-1])

In [ ]:
energies = []
for i in range(num_states):
    jth_energy = []
    for j in range(len(callback_dict["energies_history"])):
        jth_energy.append(callback_dict["energies_history"][j][i])
    energies.append(jth_energy)
    
print("Final energy outcomes:", callback_dict["energies_history"][-1])

# true_energies = [-2, 0, 0, 2] # cyclobutadiene
true_energies = [-1.618, -0.618, 0.618, 1.618] # 1,3-butadiene

fig, ax = plt.subplots(1, 1, figsize=(7,5))
colors = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red']
[ax.plot(range(callback_dict["iters"]), energies[i], color=colors[i], label=np.binary_repr(i, num_qubits)) for i in range(num_states)]
[ax.plot(range(callback_dict["iters"]), np.zeros(callback_dict["iters"]) + true_energies[i], '--', color=colors[i]) for i in range(num_states)]
ax.set_xlabel("Iterations")
ax.set_ylabel("Energy")
ax.set_ybound(-2.0, 2.0)
y_ticks = [2.0, 1.5, 1.0, 0.5, 0, -0.5, -1.0, -1.5, -2.0]
y_tick_labels = [
    r"$\alpha - 2.0\beta$", r"$\alpha - 1.5\beta$", r"$\alpha - 1.0\beta$", r"$\alpha - 0.5\beta$",
    r"$\alpha + 0.0\beta$", r"$\alpha + 0.5\beta$", r"$\alpha + 1.0\beta$", r"$\alpha + 1.5\beta$", r"$\alpha + 2.0\beta$"
]
ax.set_yticks(y_ticks ,y_tick_labels)
ax.legend(loc="lower left", bbox_to_anchor=(0, 0))

# plt.savefig('text.png', bbox_inches='tight')

print("Final parameters:", callback_dict["prev_vector"])

In [10]:
qc = ansatz_list[1].assign_parameters(callback_dict['prev_vector'])

In [ ]:
Statevector(qc)